### GeoJSON과 Choropleth Map - 대한민국 광역시/도.

In [ ]:
import json                                       # json = JavaScript Object Notation.
import pandas as pd
import numpy as np
import folium
from ipywidgets import interact                   # Ineractive 시각화 기능.
from sklearn.preprocessing import MinMaxScaler    # scikit-learn 라이브러리의 전처리 기능.

#### 1. GeoJSON 파일을 읽어와서 탐색한다:

In [ ]:
# 지리정보를 담은 GeoJSON 파일을 읽어온다.
f = open('../data/geo_korea_provinces.json', encoding='utf8')
geo_korea_provinces = json.load(f)
f.close()

In [ ]:
# geo_korea_provinces을 한번 훑어본다.
geo_korea_provinces

In [ ]:
# 광역시/도의 이름을 가져와서 본다.
my_provinces = []
for a_dict in geo_korea_provinces['features']:                            # List of dictionary objects.
    my_provinces.append(a_dict['properties']['name'])           # 광역시/도.
print(my_provinces)

In [ ]:
# Geometry의 type을 출력해 본다.
# Polygon과 MultiPolygon이 섞여 있다!
my_geo_types  = []
for a_dict in geo_korea_provinces['features']:                            # List of dictionary objects.
    my_geo_types.append(a_dict['geometry']['type'])             
print(my_geo_types)

#### 2. Choropleth Map 생성:

2.1. 데이터 전처리:

In [ ]:
# 인구 정보 DataFrame을 읽어 온다.
my_data = pd.read_csv("../data/data_provinces_population.csv",header="infer",encoding="utf8")
my_data.head(3)

In [ ]:
# 인구를 로그 스케일로 변환한 새로운 column을 만들어 본다.
my_data['인구_LOG'] = np.log(my_data['인구'])
my_data.head(3)

In [ ]:
# '시도'는 index로 들어가야 하며, 수치형 변수는 MinMaxScale 해주어야 한다.
my_columns = ['인구','인구_LOG']
my_scaler = MinMaxScaler()
X_scaled = my_scaler.fit_transform(my_data[my_columns])
my_data_scaled = pd.DataFrame(data=X_scaled, columns = my_columns, index = my_data['시도'])
my_data_scaled.head(5)

2.2. Choropleth Map 출력:

In [ ]:
# 인구 데이터로 채색한다.
my_map=folium.Map(location = [36.0, 127.5],  tiles="Stamen Toner", zoom_start=6, width='50%', height='80%')       
folium.Choropleth(geo_data=geo_korea_provinces,
                  data=my_data_scaled['인구_LOG'],   # Series 또는 Dictionary. '인구' 또는 '인구_LOG' 선택.
                  fill_opacity=0.5,
                  fill_color = "BuPu",
                  line_weight=0.5,
                  line_color='blue',
                  line_opacity=0.4,
                  legend_name='Population',
                  key_on='feature.properties.name'            
                  ).add_to(my_map)
my_map

*fill_color로 가능한 값들:* <br>
- Sequential: 'BuGn', 'BuPu','GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu','YlOrBr','YlOrRd'.
- Diverging: 'BrBg', 'PiYG', 'PRGn', 'PuOr', 'RdBu', 'RdGy', 'RdYlBu', 'RdYlGn', 'Spectral'.
- Qualitative: 'Accent', 'Qualitative', 'Dark2', 'Paired', 'Pastel1', 'Pastel2', 'Set1', 'Set2', 'Set3'.

#### 3. Interactive 시각화:

In [ ]:
# Decorator를 사용하는 방법.
my_columns = ['인구','인구_LOG']
my_colors = ['BuGn', 'BuPu','GnBu', 'OrRd', 'PuBu', 'PuBuGn', 'PuRd', 'RdPu','YlGn', 'YlGnBu','YlOrBr','YlOrRd']

@interact( data_column = my_columns, color = my_colors, zoom=(4,8) )
def choropleth_map(data_column='인구', color='BuGn', zoom=6):
    my_map=folium.Map(location = [36.0, 127.5],  tiles="Stamen Toner", zoom_start=zoom, width='50%', height='80%')
    folium.Choropleth(geo_data=geo_korea_provinces,
                  data=my_data_scaled[data_column],      
                  fill_opacity=0.5,
                  fill_color=color,
                  line_weight=0.5,
                  line_color='orange',
                  line_opacity=0.5,
                  key_on='feature.properties.name' ).add_to(my_map)   # 또는 key_on='feature.properties.name'
    return my_map

In [ ]:
# 또다른 방법.
res = interact(choropleth_map, data_column = my_columns, color = my_colors, zoom=(4,8) )